In [131]:
import numpy as np
import cv2 as cv
import random 
import math



In [192]:
number_of_correspondences=8
maximum_number_of_correspondences = 8
pws = np.zeros((3*maximum_number_of_correspondences))
us = np.zeros((2*maximum_number_of_correspondences))
alphas = np.zeros((4*maximum_number_of_correspondences))
pcs = np.zeros((3*maximum_number_of_correspondences))
uc = 320
vc = 240
fu = 800
fv = 800
cws=np.zeros((4,3))
ccs=np.zeros((4,3))


def set_internal_parameters(uc1,vc1,fu1,fv1):
    uc = uc1
    vc = vc1
    fu = fu1
    fv = fv1



def reset_correspondences(number_of_correspondences):
    number_of_correspondences = 0

def add_correspondence(X,Y,Z,u,v,n):
    pws[3 * n   ] = X
    pws[3 * n + 1] = Y
    pws[3 * n + 2] = Z
    us[2 * n    ] = u
    us[2 * n + 1] = v
    #number_of_correspondences+=1


def choose_control_points():
    # Take C0 as the reference points centroid:
    cws[0][0] = cws[0][1] = cws[0][2] = 0
    for i in range(number_of_correspondences) :
        for j in range(3):
            cws[0][j] += pws[3 * i + j]
            
    for j in range(3):
        cws[0][j] /= number_of_correspondences


    # Take C1, C2, and C3 from PCA on the reference points:
    PW0 = np.zeros((number_of_correspondences,3))
    PW0tPW0=np.zeros((3,3))
    DC=np.zeros((3,1))
    UCT=np.zeros((3,3))
    #CvMat PW0tPW0 = cvMat(3, 3, CV_64F, pw0tpw0)
    #CvMat DC      = cvMat(3, 1, CV_64F, dc)
    #CvMat UCt     = cvMat(3, 3, CV_64F, uct)

    for i in range(number_of_correspondences):
        for j in range(3):
            PW0[i][j] = pws[3 * i + j] - cws[0][j]

    #mulTransposed(PW0,PW0tPW0, 1)
    PW0tPW0=np.dot(PW0.transpose(),PW0)
    cv.SVDecomp(PW0tPW0,DC,UCT, 0, cv.SVD_MODIFY_A or cv.SVD_U_T)
    #print(number_of_correspondences)
    for i in range(1,4):
        k = math.sqrt(DC[i - 1] / number_of_correspondences)
        for j in range(3):
            cws[i][j] = cws[0][j] + k * UCT[(i - 1)][j]


def compute_barycentric_coordinates():
    
    CC = np.zeros((3, 3))
    CC_inv = np.zeros((3, 3))

    for i in range(3):
        for j in range(1,4):
            CC[i][j - 1] = cws[j][i] - cws[0][i]

    cv.invert(CC,CC_inv )
    ci = CC_inv
    for i in range(number_of_correspondences):
        pi = pws[3*i:]
        a = alphas[4*i:]

        for j in range(3):
            a[1 + j] = ci[j][0] * (pi[0] - cws[0][0])+ci[j][1] * (pi[1] - cws[0][1]) +ci[j][2] * (pi[2] - cws[0][2])
        a[0] = 1.0 - a[1] - a[2] - a[3]

def fill_M(M,row,a,u,v):
    
    M1 = M[row,:]
    M2 = M[row+1,:]

    for i in range(4):
        M1[3 * i    ] = a[i] * fu
        M1[3 * i + 1] = 0.0
        M1[3 * i + 2] = a[i] * (uc - u)

        M2[3 * i    ] = 0.0
        M2[3 * i + 1] = a[i] * fv
        M2[3 * i + 2] = a[i] * (vc - v)
    return M1,M2
        

def compute_ccs(betas,ut):

    for i in range(4):
        ccs[i][0] = ccs[i][1] = ccs[i][2] = 0.0

    for i in range(4): 
        v = ut[11 - i,:]
        for j in range(4):
            for k in range(3):
                ccs[j][k] += betas[i] * v[3 * j + k]

def compute_pcs():

    for i in range(number_of_correspondences):
        a = alphas[4*i:]
        pc = pcs[3*i:]

        for j in range(3):
            pc[j] = a[0]*ccs[0][j] + a[1] * ccs[1][j] + a[2] * ccs[2][j] + a[3] * ccs[3][j]

def compute_pose(R,t):
    choose_control_points()
    compute_barycentric_coordinates()

    M = np.zeros((2 * number_of_correspondences,12))
    #print(M)
    for i in range(number_of_correspondences):
        M[2*i],M[2*i+1]=fill_M(M, 2 * i, alphas[4*i:], us[2*i], us[2 * i + 1])
    #print(M)
  #double mtm[12 * 12], d[12], ut[12 * 12]
    MtM = np.zeros((12, 12))
    D = np.zeros((12,1))
    UT  = np.zeros((12, 12))

    #mulTransposed(M, MtM, 1)
    MtM=np.dot(M.transpose(),M)
    #print(MtM)
    cv.SVDecomp(MtM, D, UT, 0, cv.SVD_MODIFY_A or cv.SVD_U_T)
    #cvReleaseMat(&M)

    L_6x10=np.zeros((6,10))
    Rho=np.zeros(6)
    #CvMat L_6x10 = cvMat(6, 10, CV_64F, l_6x10)
    #CvMat Rho    = cvMat(6,  1, CV_64F, rho)

    compute_L_6x10(UT, L_6x10)
    compute_rho(Rho)

    Betas=np.zeros((4,4))
    rep_errors=np.zeros(4)
    Rs=np.zeros((4,3,3))
    ts=np.zeros((4,3))
    
    find_betas_approx_1(L_6x10, Rho, Betas[1])
    gauss_newton(L_6x10, Rho, Betas[1])
    rep_errors[1] = compute_R_and_t(UT, Betas[1], Rs[1], ts[1])

    find_betas_approx_2(L_6x10, Rho, Betas[2])
    gauss_newton(L_6x10, Rho, Betas[2])
    rep_errors[2] = compute_R_and_t(UT, Betas[2], Rs[2], ts[2])

    find_betas_approx_3(L_6x10, Rho, Betas[3])
    gauss_newton(L_6x10, Rho, Betas[3])
    rep_errors[3] = compute_R_and_t(UT, Betas[3], Rs[3], ts[3])

    N = 1
    if (rep_errors[2] < rep_errors[1]): N = 2
    if (rep_errors[3] < rep_errors[N]): N = 3

    copy_R_and_t(Rs[N], ts[N], R, t)

    return R,t,rep_errors[N]


def copy_R_and_t(R_src, t_src,R_dst, t_dst):

    for i in range(3):
        for j in range(3):
            R_dst[i][j] = R_src[i][j]
        t_dst[i] = t_src[i]

        
def dist2( p1,  p2):
    return (p1[0] - p2[0]) * (p1[0] - p2[0]) +(p1[1] - p2[1]) * (p1[1] - p2[1]) +(p1[2] - p2[2]) * (p1[2] - p2[2])


def dot(v1, v2):
    return v1[0] * v2[0] + v1[1] * v2[1] + v1[2] * v2[2]


def reprojection_error(R,t):

    sum2 = 0.0

    for i in range(number_of_correspondences):
        pw = pws[3 * i:]
        Xc = dot(R[0], pw) + t[0]
        Yc = dot(R[1], pw) + t[1]
        inv_Zc = 1.0 / (dot(R[2], pw) + t[2])
        ue = uc + fu * Xc * inv_Zc
        ve = vc + fv * Yc * inv_Zc
        u = us[2 * i]
        v = us[2 * i + 1]

        sum2 += math.sqrt( (u - ue) * (u - ue) + (v - ve) * (v - ve) )
  
    return sum2 / number_of_correspondences


def estimate_R_and_t(R,t):

    pc0=np.zeros(3)
    pw0=np.zeros(3)

    pc0[0] = pc0[1] = pc0[2] = 0.0
    pw0[0] = pw0[1] = pw0[2] = 0.0

    for i in range(number_of_correspondences):
        pc = pcs[3*i:]
        pw = pws[3*i:]
    
        for j in range(3):
            pc0[j] += pc[j]
            pw0[j] += pw[j]
    
  
    for j in range(3):
        pc0[j] /= number_of_correspondences
        pw0[j] /= number_of_correspondences
  

    #double abt[3 * 3], abt_d[3], abt_u[3 * 3], abt_v[3 * 3]
    ABt   = np.zeros((3, 3))
    ABt_D = np.zeros((3, 1))
    ABt_U = np.zeros((3, 3))
    ABt_V = np.zeros((3, 3))

    #cv.SetZero(ABt)
    for i in range(number_of_correspondences):
        pc = pcs[3 * i:]
        pw = pws[3 * i:]

        for j in range(3):
            ABt[j][0] += (pc[j] - pc0[j]) * (pw[0] - pw0[0])
            ABt[j][1] += (pc[j] - pc0[j]) * (pw[1] - pw0[1])
            ABt[j][2] += (pc[j] - pc0[j]) * (pw[2] - pw0[2])
    
  

    cv.SVDecomp(ABt, ABt_D, ABt_U, ABt_V, cv.SVD_MODIFY_A)

    for i in range(3):
        for j in range(3):
            R[i][j] = dot(ABt_U[i,], ABt_V[j,])

    det =R[0][0] * R[1][1] * R[2][2] + R[0][1] * R[1][2] * R[2][0] + R[0][2] * R[1][0] * R[2][1] -R[0][2] * R[1][1] * R[2][0] - R[0][1] * R[1][0] * R[2][2] - R[0][0] * R[1][2] * R[2][1]

    if det < 0 :
        R[2][0] = -R[2][0]
        R[2][1] = -R[2][1]
        R[2][2] = -R[2][2]
  

    t[0] = pc0[0] - dot(R[0], pw0)
    t[1] = pc0[1] - dot(R[1], pw0)
    t[2] = pc0[2] - dot(R[2], pw0)


def print_pose(R,t):

    print(R[0][0]) 
    print(" ") 
    print(R[0][1])
    print(" ")
    print(R[0][2])
    print(" ")
    print(t[0])
    print('\n')
                 
    print(R[1][0])
    print(" ")
    print(R[1][1])
    print(" ")
    print(R[1][2])
    print(" ")
    print(t[1])
    print('\n')
                 
    print(R[2][0]) 
    print(" ") 
    print(R[2][1])
    print(" ")
    print(R[2][2])
    print(" ")
    print(t[2])
    print('\n')

    
def solve_for_sign():

    if pcs[2] < 0.0: 
        for i in range(4):
            for j in range(3):
                ccs[i][j] = -ccs[i][j]

    for i in range(number_of_correspondences): 
        pcs[3 * i    ] = -pcs[3 * i]
        pcs[3 * i + 1] = -pcs[3 * i + 1]
        pcs[3 * i + 2] = -pcs[3 * i + 2]
    

def compute_R_and_t(ut,betas,R,t):

    compute_ccs(betas, ut)
    compute_pcs()

    solve_for_sign()

    estimate_R_and_t(R, t)

    return reprojection_error(R, t)


# betas10        = [B11 B12 B22 B13 B23 B33 B14 B24 B34 B44]
# betas_approx_1 = [B11 B12     B13         B14]

def find_betas_approx_1(L_6x10,Rho,betas):

    L_6x4=np.zeros((6,4))
    b4=np.zeros([4])
    #L_6x4 = cvMat(6, 4, CV_64F, l_6x4)
    #CvMat B4    = cvMat(4, 1, CV_64F, b4)

    for i in range(6): 
        L_6x4[i][0]=(L_6x10[i][0])
        L_6x4[i][1]=(L_6x10[i][1])
        L_6x4[i][2]=(L_6x10[i][3])
        L_6x4[i][3]=(L_6x10[i][6])
        
  

    cv.solve(L_6x4, Rho, b4, cv.DECOMP_SVD)

    if b4[0] < 0: 
        betas[0] = math.sqrt(-b4[0])
        betas[1] = -b4[1] / betas[0]
        betas[2] = -b4[2] / betas[0]
        betas[3] = -b4[3] / betas[0]
    else :
        betas[0] = math.sqrt(b4[0])
        betas[1] = b4[1] / betas[0]
        betas[2] = b4[2] / betas[0]
        betas[3] = b4[3] / betas[0]
  


# betas10        = [B11 B12 B22 B13 B23 B33 B14 B24 B34 B44]
# betas_approx_2 = [B11 B12 B22                            ]

def find_betas_approx_2(L_6x10,Rho,betas):

  #double l_6x3[6 * 3], b3[3]
    L_6x3  = np.zeros((6,3))
    b3     = np.zeros((3,1))

    for i in range(6):
        L_6x3[i][0]=(L_6x10[i][0])
        L_6x3[i][1]=(L_6x10[i][1])
        L_6x3[i][2]=(L_6x10[i][2])
        
  

    cv.solve(L_6x3, Rho, b3, cv.DECOMP_SVD)

    if b3[0] < 0: 
        betas[0] = math.sqrt(-b3[0])
        betas[1] = math.sqrt(-b3[2]) if(b3[2] < 0) else 0.0
    else: 
        betas[0] = math.sqrt(b3[0])
        betas[1] = math.sqrt(b3[2]) if(b3[2] > 0) else 0.0
  

    if b3[1] < 0:
        betas[0] = -betas[0]

    betas[2] = 0.0
    betas[3] = 0.0


# betas10        = [B11 B12 B22 B13 B23 B33 B14 B24 B34 B44]
# betas_approx_3 = [B11 B12 B22 B13 B23                    ]

def find_betas_approx_3(L_6x10,Rho,betas):

  #double l_6x5[6 * 5], b5[5]
    L_6x5 = np.zeros((6, 5))
    b5    = np.zeros((5, 1))

    for i in range(6):
        L_6x5[i][0]=(L_6x10[i][0])
        L_6x5[i][1]=(L_6x10[i][1])
        L_6x5[i][2]=(L_6x10[i][2])
        L_6x5[i][3]=(L_6x10[i][3])
        L_6x5[i][4]=(L_6x10[i][4])
        
        
  

    cv.solve(L_6x5, Rho, b5, cv.DECOMP_SVD)

    if (b5[0] < 0): 
        betas[0] = math.sqrt(-b5[0])
        betas[1] = math.sqrt(-b5[2]) if(b5[2] < 0) else 0.0
    else: 
        betas[0] = math.sqrt(b5[0])
        betas[1] = math.sqrt(b5[2]) if(b5[2] > 0) else 0.0
  
    if (b5[1] < 0):
        betas[0] = -betas[0]
        betas[2] = b5[3] / betas[0]
        betas[3] = 0.0


def compute_L_6x10(ut,l_6x10):

    v=np.zeros([4,12])

    v[0] = ut[11,:]
    v[1] = ut[10,:]
    v[2] = ut[9,:]
    v[3] = ut[8,:]

    dv=np.zeros((4,6,3))

    for i in range(4): 
        a = 0
        b = 1
        for j in range(6): 
            dv[i][j][0] = v[i][3 * a    ] - v[i][3 * b]
            dv[i][j][1] = v[i][3 * a + 1] - v[i][3 * b + 1]
            dv[i][j][2] = v[i][3 * a + 2] - v[i][3 * b + 2]

            b=b+1
            if (b > 3) :
                a=a+1
                b = a + 1
      
    
  

    for i in range(6): 
        row = l_6x10[i,:]

        row[0] =        dot(dv[0][i], dv[0][i])
        row[1] = 2.0 * dot(dv[0][i], dv[1][i])
        row[2] =        dot(dv[1][i], dv[1][i])
        row[3] = 2.0 * dot(dv[0][i], dv[2][i])
        row[4] = 2.0 * dot(dv[1][i], dv[2][i])
        row[5] =        dot(dv[2][i], dv[2][i])
        row[6] = 2.0 * dot(dv[0][i], dv[3][i])
        row[7] = 2.0 * dot(dv[1][i], dv[3][i])
        row[8] = 2.0 * dot(dv[2][i], dv[3][i])
        row[9] =        dot(dv[3][i], dv[3][i])


def compute_rho(rho):

    rho[0] = dist2(cws[0], cws[1])
    rho[1] = dist2(cws[0], cws[2])
    rho[2] = dist2(cws[0], cws[3])
    rho[3] = dist2(cws[1], cws[2])
    rho[4] = dist2(cws[1], cws[3])
    rho[5] = dist2(cws[2], cws[3])


def compute_A_and_b_gauss_newton(l_6x10,rho,betas,A,b):

    for i in range(6):
        rowL = l_6x10[i,:]
        rowA = A[i,:]

        rowA[0] = 2*rowL[0]*betas[0] +     rowL[1] * betas[1] +     rowL[3] * betas[2] +     rowL[6] * betas[3]
        rowA[1] = rowL[1] * betas[0] + 2 * rowL[2] * betas[1] +     rowL[4] * betas[2] +     rowL[7] * betas[3]
        rowA[2] = rowL[3] * betas[0] +     rowL[4] * betas[1] + 2 * rowL[5] * betas[2] +     rowL[8] * betas[3]
        rowA[3] = rowL[6] * betas[0] +     rowL[7] * betas[1] +     rowL[8] * betas[2] + 2 * rowL[9] * betas[3]

        b[i][0]= (rho[i] -(rowL[0] * betas[0] * betas[0] +rowL[1] * betas[0] * betas[1] +rowL[2] * betas[1] * betas[1] +rowL[3] * betas[0] * betas[2] +rowL[4] * betas[1] * betas[2] +rowL[5] * betas[2] * betas[2] +rowL[6] * betas[0] * betas[3] +rowL[7] * betas[1] * betas[3] +rowL[8] * betas[2] * betas[3] +rowL[9] * betas[3] * betas[3]))
    

def gauss_newton(L_6x10,Rho,betas):

    
    n_iterations=5;
    #a=np.zeros([6*4]), b[6], x[4]
    A=np.zeros((6,4))
    b =np.zeros((6,1))
    
    
    for k in range(n_iterations): 
        compute_A_and_b_gauss_newton(L_6x10, Rho,betas, A, b)
        dbeta=np.dot(np.linalg.inv(np.dot(A.transpose(),A)),np.dot(A.transpose(),b))
        for i in range(4):
            betas[i]+=dbeta[i]
    



def relative_error(rot_err,transl_err,Rtrue,ttrue,Rest,test):

    qtrue=np.zeros([4])
    qest=np.zeros([4])

    mat_to_quat(Rtrue, qtrue)
    mat_to_quat(Rest, qest)

    rot_err1 = math.sqrt((qtrue[0] - qest[0]) * (qtrue[0] - qest[0]) +(qtrue[1] - qest[1]) * (qtrue[1] - qest[1]) +(qtrue[2] - qest[2]) * (qtrue[2] - qest[2]) +(qtrue[3] - qest[3]) * (qtrue[3] - qest[3]) ) /math.sqrt(qtrue[0] * qtrue[0] + qtrue[1] * qtrue[1] + qtrue[2] * qtrue[2] + qtrue[3] * qtrue[3])

    rot_err2 = math.sqrt((qtrue[0] + qest[0]) * (qtrue[0] + qest[0]) +(qtrue[1] + qest[1]) * (qtrue[1] + qest[1]) +(qtrue[2] + qest[2]) * (qtrue[2] + qest[2]) +(qtrue[3] + qest[3]) * (qtrue[3] + qest[3]) ) /math.sqrt(qtrue[0] * qtrue[0] + qtrue[1] * qtrue[1] + qtrue[2] * qtrue[2] + qtrue[3] * qtrue[3])

    rot_err = min(rot_err1, rot_err2)

    transl_err = math.sqrt((ttrue[0] - test[0]) * (ttrue[0] - test[0]) +(ttrue[1] - test[1]) * (ttrue[1] - test[1]) +(ttrue[2] - test[2]) * (ttrue[2] - test[2])) /math.sqrt(ttrue[0] * ttrue[0] + ttrue[1] * ttrue[1] + ttrue[2] * ttrue[2])


def mat_to_quat(R,q):

    tr = R[0][0] + R[1][1] + R[2][2]
    n4=0

    if (tr > 0.0): 
        q[0] = R[1][2] - R[2][1]
        q[1] = R[2][0] - R[0][2]
        q[2] = R[0][1] - R[1][0]
        q[3] = tr + 1.0
        n4 = q[3]
    elif ( (R[0][0] > R[1][1]) and (R[0][0] > R[2][2]) ):
        q[0] = 1.0 + R[0][0] - R[1][1] - R[2][2]
        q[1] = R[1][0] + R[0][1]
        q[2] = R[2][0] + R[0][2]
        q[3] = R[1][2] - R[2][1]
        n4 = q[0]
    elif (R[1][1] > R[2][2]):
        q[0] = R[1][0] + R[0][1]
        q[1] = 1.0 + R[1][1] - R[0][0] - R[2][2]
        q[2] = R[2][1] + R[1][2]
        q[3] = R[2][0] - R[0][2]
        n4 = q[1]
    else: 
        q[0] = R[2][0] + R[0][2]
        q[1] = R[2][1] + R[1][2]
        q[2] = 1.0 + R[2][2] - R[0][0] - R[1][1]
        q[3] = R[0][1] - R[1][0]
        n4 = q[2]
  
    scale = 0.5 / (math.sqrt(n4))

    q[0] *= scale
    q[1] *= scale
    q[2] *= scale
    q[3] *= scale


In [193]:
n = 8
noise = 10

def rand(min,max):

    return random.uniform(min,max)


def random_pose(R,t):

    range = 1

    phi   = rand(0, range * 3.14159 * 2)
    theta = rand(0, range * 3.14159)
    psi   = rand(0, range * 3.14159 * 2)

    R[0][0] = math.cos(psi) * math.cos(phi) - math.cos(theta) * math.sin(phi) * math.sin(psi)
    R[0][1] = math.cos(psi) * math.sin(phi) + math.cos(theta) * math.cos(phi) * math.sin(psi)
    R[0][2] = math.sin(psi) * math.sin(theta)

    R[1][0] = -math.sin(psi) * math.cos(phi) - math.cos(theta) * math.sin(phi) * math.cos(psi)
    R[1][1] = -math.sin(psi) * math.sin(phi) + math.cos(theta) * math.cos(phi) * math.cos(psi)
    R[1][2] = math.cos(psi) * math.sin(theta)

    R[2][0] = math.sin(theta) * math.sin(phi)
    R[2][1] = -math.sin(theta) * math.cos(phi)
    R[2][2] = math.cos(theta)

    t[0] = 0.0
    t[1] = 0.0
    t[2] = 6.0
    return R,t


def  random_point(Xw,Yw,Zw):

    theta = rand(0, 3.14159)
    phi = rand(0, 2 * 3.14159)
    R = rand(0, +2)

    Xw =  math.sin(theta) * math.sin(phi) * R
    Yw = -math.sin(theta) * math.cos(phi) * R
    Zw =  math.cos(theta) * R
    return Xw,Yw,Zw
    #print(Xw)


def  project_with_noise( R, t,Xw,Yw,Zw,u, v):

    Xc = R[0][0] * Xw + R[0][1] * Yw + R[0][2] * Zw + t[0]
    Yc = R[1][0] * Xw + R[1][1] * Yw + R[1][2] * Zw + t[1]
    Zc = R[2][0] * Xw + R[2][1] * Yw + R[2][2] * Zw + t[2]

    nu = rand(-noise, +noise)
    nv = rand(-noise, +noise)
    u = uc + fu * Xc / Zc + nu
    v = vc + fv * Yc / Zc + nv
    return u,v
    
    #MAIN PROGRAM

#set_internal_parameters(uc, vc, fu, fv)

#set_maximum_number_of_correspondences(n)

R_true=np.zeros([3,3])
t_true=np.zeros([3])
R_true, t_true=random_pose(R_true, t_true)

#reset_correspondences(number_of_correspondences)
number_of_correspondences=8

for i in range(n): 
    Xw=0.0
    Yw=0.0
    Zw=0.0
    u=0.0
    v=0.0
    #print(i)
    #print(Xw)
    Xw,Yw,Zw=random_point(Xw, Yw, Zw)
    #print(Xw)
    u,v=project_with_noise(R_true, t_true, Xw, Yw, Zw, u, v)
    add_correspondence(Xw, Yw, Zw, u, v,i)



R_est=np.zeros([3,3])
t_est=np.zeros([3])
R_est,t_est,err2 = compute_pose(R_est, t_est)
rot_err=0
transl_err=0


relative_error(rot_err, transl_err, R_true, t_true, R_est, t_est)
#cout << ">>> Reprojection error: " << err2 << endl
#cout << ">>> rot_err: " << rot_err << ", transl_err: " << transl_err << endl
#cout << endl
#cout << "'True reprojection error':"<< PnP.reprojection_error(R_true, t_true) << endl
#cout << endl
print("True pose:")
print('\n')
print_pose(R_true, t_true)
print('\n')
print("found pose:")
print('\n')
print_pose(R_est, t_est)


True pose:


0.6877089506163899
 
-0.7193096617691213
 
0.09823446303459972
 
0.0


-0.7043443297774418
 
-0.6282790345169852
 
0.3304005446375224
 
0.0


-0.17594165041987692
 
-0.29641029886290626
 
-0.9387147971431645
 
6.0




found pose:


0.27216023868203854
 
-0.4779231611776148
 
-0.8351755842279702
 
0.3238969505806009


-0.699979963226512
 
-0.6938861744222157
 
0.16896753542356355
 
0.7711248898825399


-0.6602702897768054
 
0.5386199299651826
 
-0.5233848636351219
 
-0.10573836046462592


